<img src="images/ublogo.png"/>

### CSE610 - Bayesian Non-parametric Machine Learning

  - Lecture Notes
  - Instructor - Varun Chandola
  - Term - Fall 2020

### Objective
The objective of this notebook is to discuss the relationship between Gaussian process based methods, discussed in the class so far, and deep-learning. In particular, we will talk about: a). the relationship between the two, b). how to combine GPs with neural networks, and c). how to make GPs behave like deep neural networks, i.e., deep Gaussian Processes.

<div class="alert alert-info">

**Note:** This material is based on a collection of recent papers on this topic, including [Deep neural networks as Gaussian Processes](https://arxiv.org/pdf/1711.00165.pdf), and [Deep Gaussian Processes](http://proceedings.mlr.press/v31/damianou13a.pdf).

</div>

### Equivalence between GP and Neural Networks

> This was established way back in 1994 by Radford Neal in this PhD thesis - *Bayesian Learning for Neural Networks*

Consider a multi-layered perceptron. For any hidden layer (indexed by $l$), we will use the following notation:
- $N_l$ - width of the layer $l$ or the number of nodes
- ${\bf x}^l$ - input to the $l^{th}$ layer (length will be $N_{l-1}$)
- ${\bf z}^l$ - output (before non-linear transformation) of the $l^{th}$ layer (length will be $N_l$) 
- ${\bf b}^l$ - bias vector for the $l^{th}$ layer (length will be $N_l$)
- $\phi(\cdot)$ - non-linear activation function
- $i$ - index of the $i^{th}$ node in the hidden layer
- $W^l$ - weight matrix at the $l^{th}$ hidden layer
  * $W^{l}_{ij}$ - The $j^{th}$ component of the weight vector for the $i^{th}$ unit in the $l^{th}$ layer

Thus, ${\bf x}^0$ will correspond to the input (${\bf x}$) to the neural network, while $\phi({\bf z}^L)$ will be the output of the neural network, where $L$ is the index of the final layer.

Consider a single-hidden layer neural network. The input to the final layer will be, ${\bf x}^1$, whose $j^{th}$ component will be:
$$
x_j^1({\bf x}) = \phi(b_j^0 + \sum_{k=1}^d W^0_{jk}x_k)
$$
Notice that we are using $x_k$ (the $k^{th}$ entry in the input vector ${\bf x}$), instead of the general term $x^0_k$.

The output of the final layer will be:
$$
z_i^1({\bf x}) = b_i^1 + \sum_{j=1}^{N_0} W^1_{ij}x^1_j
$$

Also note that we are writing these terms as a function of the input $({\bf x})$, since their values depend on the input to the neural network.

We assume that the weight and bias parameters for layer $l$ are independent and randomly drawn from a Gaussian with zero mean and variance as $\sigma^2_w/N_l$ and $\sigma^2_b$, respectively.

> **Observation 1**: The post-activations, $x^1_j({\bf x})$ and $x^1_{j'}({\bf x})$ are independent of each other. This is true because the weight and bias parameters are assumed to be i.i.d.

> **Observation 2**: $z^1_i({\bf x})$ will be Gaussian distributed. This is true because $z^1_i({\bf x})$ is a sum of i.i.d. terms. And, the **Central Limit Theorem** dictates that in the limit of infinite width, i.e., $N_1 \rightarrow \infty$, the sum will be a random variable with a Gaussian distribution.

Consider the values of $z^1_i$ for several inputs, $\{{\bf x}_1, {\bf x}_2,\ldots,{\bf x}_k\}$.
> **Observation 3**: The finite collection, $\{z^1_i({\bf x}_1), z^1_i({\bf x}_2),\ldots,z^1_i({\bf x}_k\})$, will have a joint multivariate Gaussian distribution.

If we consider $z^1_i(\cdot)$ as a function of ${\bf x}$, then the above observation is equivalent to a Gaussian process.

> **Observation 4**: $z^1_i({\bf x}) \sim \mathcal{GP}(\mu^1({\bf x}),k^1({\bf x},{\bf x}'))$

Note that the GP prior is independent of $i$.

Since the parameters have zero mean, the mean function will also be 0, i.e.,
$$
\mu^1({\bf x}) \equiv \mathbb{E}[z_i^1({\bf x})] = 0
$$
The covariance function can also be derived from the above expressions as:
$$
k^1({\bf x},{\bf x}') \equiv \mathbb{E}[z_i^1({\bf x})z_i^1({\bf x}')] = \sigma^2_b + \sigma^2_w\mathbb{E}[x_i^1({\bf x})x_i^1({\bf x}')] \equiv \sigma^2_b + \sigma^2_wC({\bf x},{\bf x}')
$$
where $C(\cdot,\cdot)$ can be analytically computed for a given activation function $\phi(\cdot)$. 

For example, as discussed in Chapter 4 of the GPML book, if $\phi(\cdot)$ is the `erf` function, i.e.,
$$
\phi(y) = erf(y) = \frac{2}{\pi}\int_0^y e^{-t^2}dt
$$
then the corresponding $C$ function is obtained as:
$$
C({\bf x},{\bf x}') = \frac{2}{\pi}\sin^{-1}\left(\frac{2\tilde{{\bf x}}^\top\Sigma \tilde{{\bf x}}'}{\sqrt{(1+2\tilde{{\bf x}}^\top\Sigma \tilde{{\bf x}})(1+2\tilde{{\bf x}}'^\top\Sigma \tilde{{\bf x}}')}}\right)
$$
where $\tilde{\bf x}'$ is the augmented input vector (including a 1 for the bias term) and $\Sigma$ is a diagonal matrix consisting of $\sigma^2_b$ and $\sigma^2_w$ on the diagonal terms.

<div class="alert alert-warning">
Above discussion shows that an infinite width single-hidden layer neural network is equivalent to a GP with a specific covariance function, which depends on the activation function used in the neural network.
</div>

This can be extended to deeper layers by induction. Thus, if the neural network has several hidden layers with infinite width, the output at each hidden layer is equivalent to a GP, where the covariance function at a given hidden layer recursively depends on the covariance function at the previous hidden layer.

$$
k^l({\bf x},{\bf x}') = \sigma^2_b + \sigma^2_wF_{\phi}(k^{l-1}({\bf x},{\bf x}'),k^{l-1}({\bf x},{\bf x}),k^{l-1}({\bf x}',{\bf x}'))
$$
where $F_{\phi}$ is a determinstic function whose form depends on the activation function $\phi$.

### What does this get us?

With the above equivalence, we can now construct a GP model, and use it as a infinite width neural network.

<img src='images/gpnn.png'/>